In [2]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
import random
import tensorflow as tf
from keras import models, layers
from keras import backend

# import numpy as np

In [2]:
mySeed = 42
# np.random.seed(mySeed)
random.seed(mySeed)
tf.random.set_seed(mySeed)
# torch.manual_seed(mySeed)

In [3]:
# Hyper parameters

num_clients = 4
batch_size = 32
total_steps = 2

lr = 0.01
# optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr, decay=lr/total_steps)
optimizer = "adam"
loss_cce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
metrics = ["accuracy"]
client_epochs = 1

swap_step = 2
n_swap_between_avg = 2

## Load Data

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize data
x_train, x_test = x_train/255, x_test/255

# OneHotEncoded Labels
oneHotEncoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
y_train = oneHotEncoder.fit_transform(y_train.reshape(-1, 1))
y_test = oneHotEncoder.transform(y_test.reshape(-1, 1))

print(f"Train x={x_train.shape}, y={y_train.shape}")
print(f"Test x={x_test.shape}, y={y_test.shape}")

Train x=(60000, 28, 28), y=(60000, 10)
Test x=(10000, 28, 28), y=(10000, 10)


In [5]:
# Remove one dimension for data
x_train = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(-1, x_test.shape[1]*x_test.shape[2])

print(f"Train x={x_train.shape}, y={y_train.shape}")
print(f"Test x={x_test.shape}, y={y_test.shape}")

Train x=(60000, 784), y=(60000, 10)
Test x=(10000, 784), y=(10000, 10)


In [6]:
def create_clients_with_data_assignment(values_list, label_list, num_clients=10, initial="client"):
    """ return: A dictionary with the customer id as the dictionary key and the value
                will be the data fragment - tuple of values and labels.
        args:
            values_list: a numpy array object with the values
            label_list: list of binarized labels (one-hot encoded)
            num_clients: number of customers (clients)
            initial: the prefix of the clients, e.g., client_1
     """

    # create list of client names
    client_names = [f"{initial}_{i+1}" for i in range(num_clients)]

    # shuffle the data
    data = list(zip(values_list, label_list))
    random.shuffle(data)

    # shard the data and split it for each customer
    size = len(data) // num_clients
    shards = [data[i: i+size]  for i in range(0, size*num_clients, size)]

    # Check if the fragment number is equal to the number of clients
    assert(len(shards) == len(client_names))

    return {client_names[i]: shards[i]  for i in range(len(client_names))}

In [7]:
def batch_data(data_shard, batch_size=32):
    """ Receives a piece of data (values, labels) from a client and creates a tensorflow Dataset object in it
        args:
            data_shard: values and labels that make up a customer's data shard
            batch_size: batch size
        return:
            data tensorflow Dataset object
    """
    #seperate shard into data and labels lists
    values, labels = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(values), list(labels)))
    return dataset.shuffle(len(labels), reshuffle_each_iteration=False).batch(batch_size)


### Explaining the function with example ###
# test = list(zip([[1, 2], [3, 4], [5, 6], [7, 8]], ["a", "b", "c", "d"]))
# print(test)
# data, label = zip(*test)
# print(data)
# print(label)

# dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
# test1 = dataset.shuffle(len(label))
# print(list(test1.as_numpy_iterator()))
# test2 = dataset.shuffle(len(label)).batch(3)
# print(list(test2.as_numpy_iterator()))
# list(test2)[0][0].shape

In [8]:
# Create clients and batched data

clients = create_clients_with_data_assignment(x_train, y_train, num_clients=num_clients, initial="client")

# Bached data with tensorflow data object
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data, batch_size)

test_batch_size = len(y_test)
# Convert test data to tensorflow data object
test_batched = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(test_batch_size)

## Training

In [9]:
class MLP:
    @staticmethod
    def build(shape, classes):
        model = models.Sequential([
            layers.Dense(100, activation="relu", input_shape=shape),
            layers.Dense(100, activation="relu"),
            layers.Dense(classes, activation="softmax"),
            ])

        return model

In [50]:
# Initialize all clients with same weight
mlp_global = MLP()
global_model = mlp_global.build(x_train.shape[1:], y_train.shape[-1])
global_weights = global_model.get_weights()
client_weights = dict.fromkeys(list(clients_batched.keys()), global_weights)

client_select = list(clients_batched.keys())

In [ ]:
def fed_swap(client):
    random_num = random.randint(0, len(client_select)-1)
    random_client = client_select[random_num]

    temp_weight = client_weights[random_client]
    client_weights[random_client] = client_weights[client]

    return temp_weight

In [ ]:
for step in total_steps:
    for client in client_select:
        mlp_local = MLP()
        local_model = mlp_local.build(x_train.shape[1:], y_train.shape[-1])
        local_model.compile(optimizer=optimizer, loss=loss_cce, metrics=metrics)

        local_model.set_weights(client_weights[client])
        local_model.fit(clients_batched[client], epochs=client_epochs, verbose=0)

        backend.clear_session()
    
    if (step % swap_step == 0) and (step % (swap_step*n_swap_between_avg) != 0):
        for client in client_select:
            client_weights[client] = fed_swap(client)
